# Install Packages

In [ ]:
!pip3 install top2vec[sentence_encoders] flask_ngrok pyngrok -q lib5c flask==0.12.2

     |████████████████████████████████| 83 kB 1.1 MB/s 


In [ ]:
!ngrok start --none --log stdout

# Store your personal ngrok auth token on colab

In [ ]:
!ngrok authtoken 5gVpa4emhkybkkg4hZFzy_4457EWNhXHhuRNEq4rEgs

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Flask App

# Imports

In [ ]:
from flask import Flask, render_template_string,render_template
from flask_ngrok import run_with_ngrok
import pandas as pd
from top2vec import Top2Vec
from urllib.parse import urlparse,  parse_qs


# Ingest dataset

In [ ]:
with pd.read_csv('/content/drive/MyDrive/models/amazon_reviews_books.csv', 
            chunksize=10000, 
            nrows = 100000,
            sep=',' , 
            #usecols=['product_title','product_parent','review_headline','review_body','review_date','star_rating'],
            usecols=['product_title','review_body','star_rating'],
            ) as reader:

  df_small = pd.DataFrame()
  for chunk in reader:
    df_small = pd.concat([df_small, chunk])


In [ ]:
df_small.head()

,product_title,review_body,star_rating
0,Sisterhood of the Traveling Pants (Book 1),this boook was a great one that you could lear...,4
1,The Bad Girl's Guide to Getting What You Want,If you are looking for something to stimulate ...,3
2,"Eisenhorn (A Warhammer 40,000 Omnibus)",never read it-a young relative idicated he lik...,4
3,Colby Conspiracy (Colby Agency),Though she is honored to be Chicago Woman of t...,5
4,The Psychology of Proof: Deductive Reasoning i...,Review based on a cursory examination by Unive...,4


# Initialize app and model

In [ ]:



app = Flask(__name__, template_folder='/content/drive/MyDrive/Colab Notebooks/templates/')
run_with_ngrok(app)

model = Top2Vec.load('/content/drive/MyDrive/models/book_recomm_model')
 
#route to homepage
@app.route('/', methods=['GET'])
def index():
  return render_template('index.html')

#route to results page
@app.route('/results/<data>', methods=['GET'])
def load_results(data):
    d = {}
    d = parse_qs(str(data))
    #print("this is :" + data)
  

    return "this is :" + data


if __name__ == "__main__":
    app.run()
    

 

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://c93a-34-83-110-212.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Nov/2022 13:22:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Nov/2022 13:22:30] "GET /favicon.ico HTTP/1.1" 404 -


In [ ]:
documents, document_scores, document_ids = model.query_documents('romance', num_docs=5)
df = df_small.loc[document_ids,['product_title','star_rating','review_body']]
df.sort_values(by=['star_rating'], ascending=False)
results = df.to_html()
results

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>product_title</th>\n      <th>star_rating</th>\n      <th>review_body</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>81382</th>\n      <td>Chill Factor: A Novel</td>\n      <td>4</td>\n      <td>I love SB as an author.  I do prefer her romance novels, but this book did not disappoint.  It was thrilling with a bit of romance thrown in for good measure.</td>\n    </tr>\n    <tr>\n      <th>2479</th>\n      <td>Veiled Promises (Zebra Debut)</td>\n      <td>5</td>\n      <td>Tracy MacNish is a name to remember for future historical romance reads. This novel has everything a good old fashion historical romance needs. The heroine &amp; hero are strong characters with complex circumstances and mean villians that weave into their relationship. You can\'t put the book down. You will have a sleepless night, so make sure you have time for this one. It is worth making the 

# Run app